In [80]:
import numpy as np
import math
import re
import pandas as pd
from bs4 import BeautifulSoup
import random
import seaborn as sns
import matplotlib.pyplot as plt
import tensorflow as tf
import os
import transformers

In [27]:
def clean_tweet(tweet):
  tweet = re.sub(r"@[A-Za-z0-9]+", ' ', tweet)
  tweet = re.sub(r"https?://[A-Za-z0-9./]+", ' ', tweet)
  tweet = re.sub(r"[^a-zA-Z.!?']", ' ', tweet)
  tweet = re.sub(r" +", ' ', tweet)
  return tweet
def remove_emoji(string):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F" # emoticons
                           u"\U0001F300-\U0001F5FF" # symbols & pictographs
                           u"\U0001F680-\U0001F6FF" # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF" # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', string)


In [28]:
data = pd.read_csv('samples/reviews.csv',
                   engine='python',
                   encoding='latin1')

In [29]:
data.head()

,reviewId,userName,userImage,content,score,thumbsUpCount,reviewCreatedVersion,at,replyContent,repliedAt,sortOrder,appId
0,gp:AOqpTOEhZuqSqqWnaKRgv-9ABYdajFUB0WugPGh-SG-...,Eric Tie,https://play-lh.googleusercontent.com/a-/AOh14...,I cannot open the app anymore,1,0,5.4.0.6,2020-10-27 21:24:41,NaN,NaN,newest,com.anydo
1,gp:AOqpTOH0WP4IQKBZ2LrdNmFy_YmpPCVrV3diEU9KGm3...,john alpha,https://play-lh.googleusercontent.com/a-/AOh14...,I have been begging for a refund from this app...,1,0,NaN,2020-10-27 14:03:28,"Please note that from checking our records, yo...",2020-10-27 15:05:52,newest,com.anydo
2,gp:AOqpTOEMCkJB8Iq1p-r9dPwnSYadA5BkPWTf32Z1azu...,Sudhakar .S,https://play-lh.googleusercontent.com/a-/AOh14...,Very costly for the premium version (approx In...,1,0,NaN,2020-10-27 08:18:40,NaN,NaN,newest,com.anydo
3,gp:AOqpTOGFrUWuKGycpje8kszj3uwHN6tU_fd4gLVFy9z...,SKGflorida@bellsouth.net DAVID S,https://play-lh.googleusercontent.com/-75aK0WF...,"Used to keep me organized, but all the 2020 UP...",1,0,NaN,2020-10-26 13:28:07,What do you find troublesome about the update?...,2020-10-26 14:58:29,newest,com.anydo
4,gp:AOqpTOHls7DW8wmDFzTkHwxuqFkdNQtKHmO6Pt9jhZE...,Louann Stoker,https://play-lh.googleusercontent.com/-pBcY_Z-...,Dan Birthday Oct 28,1,0,5.6.0.7,2020-10-26 06:10:50,NaN,NaN,newest,com.anydo


In [30]:
data.columns

Index(['reviewId', 'userName', 'userImage', 'content', 'score',
       'thumbsUpCount', 'reviewCreatedVersion', 'at', 'replyContent',
       'repliedAt', 'sortOrder', 'appId'],
      dtype='object')

In [31]:
data_df = data[['content', 'score']]
data_df.head()


,content,score
0,I cannot open the app anymore,1
1,I have been begging for a refund from this app...,1
2,Very costly for the premium version (approx In...,1
3,"Used to keep me organized, but all the 2020 UP...",1
4,Dan Birthday Oct 28,1


In [32]:
data_df.tail()

,content,score
12490,"I really like the planner, it helps me achieve...",5
12491,ð****ð,5
12492,Very useful apps. You must try it,5
12493,Would pay for this if there were even more add...,5
12494,Sooow good,5


In [33]:
data_df['score'].value_counts()

5    2879
4    2775
1    2506
2    2344
3    1991
Name: score, dtype: int64

In [34]:
clean_df = [clean_tweet(remove_emoji(tweet)) for tweet in data_df.content]

In [35]:
clean_df[0:4]

['I cannot open the app anymore',
 'I have been begging for a refund from this app for over a month and nobody is replying me',
 'Very costly for the premium version approx Indian Rupees per year . Better to download the premium version of this app from apkmos website and use it. Microsoft to do list app is far more better.',
 "Used to keep me organized but all the UPDATES have made a mess of things !!! Y cudn't u leave well enuf alone ??? Guess ur techies feel the need to keep making changes to justify continuing to collect their salary !!! "]

In [36]:
data_labels = data.score.values
data_labels

array([1, 1, 1, ..., 5, 5, 5], dtype=int64)

In [124]:
#from transformers import BertTokenizer
from transformers import BertTokenizer, BertForPreTraining, AutoModel, AutoTokenizer

#tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')
tokenizer.tokenize('My dog likes strawberries.')

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

['my', 'dog', 'likes', 'straw', '##berries', '.']

In [105]:
from sklearn.model_selection import train_test_split
import random

data_inputs = [sent for sent in clean_df if len(sent.split()) > 2]
print(data_inputs[0])

I cannot open the app anymore


In [172]:
data_with_len = [[sent, data_labels[i]
                 ,len(sent.split())]
                 for i, sent in enumerate(data_inputs)]
random.shuffle(data_with_len)
data_with_len.sort(key=lambda x: x[2])
sorted_all = []
for sent_lab in data_with_len:
     if sent_lab[2] > 2:
        data = tokenizer(sent_lab[0], max_length=512, truncation=True, padding='max_length')
        sorted_all.append(([data['input_ids'],data['token_type_ids'],data['attention_mask']],sent_lab[1]))
print(sorted_all[0])

([[101, 4907, 5798, 13323, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [173]:
all_dataset = tf.data.Dataset.from_generator(lambda: sorted_all,
                                             output_types = (tf.int32, tf.int32))
for i in all_dataset:
    print(i)


(<tf.Tensor: shape=(3, 512), dtype=int32, numpy=
array([[ 101, 4907, 5798, ...,    0,    0,    0],
       [   0,    0,    0, ...,    0,    0,    0],
       [   1,    1,    1, ...,    0,    0,    0]])>, <tf.Tensor: shape=(), dtype=int32, numpy=1>)
(<tf.Tensor: shape=(3, 512), dtype=int32, numpy=
array([[ 101, 2200, 6179, ...,    0,    0,    0],
       [   0,    0,    0, ...,    0,    0,    0],
       [   1,    1,    1, ...,    0,    0,    0]])>, <tf.Tensor: shape=(), dtype=int32, numpy=2>)
(<tf.Tensor: shape=(3, 512), dtype=int32, numpy=
array([[ 101, 3733, 2000, ...,    0,    0,    0],
       [   0,    0,    0, ...,    0,    0,    0],
       [   1,    1,    1, ...,    0,    0,    0]])>, <tf.Tensor: shape=(), dtype=int32, numpy=3>)
(<tf.Tensor: shape=(3, 512), dtype=int32, numpy=
array([[  101, 19119,  2012, ...,     0,     0,     0],
       [    0,     0,     0, ...,     0,     0,     0],
       [    1,     1,     1, ...,     0,     0,     0]])>, <tf.Tensor: shape=(), dtype=int32, nump

(<tf.Tensor: shape=(3, 512), dtype=int32, numpy=
array([[ 101, 3722, 1998, ...,    0,    0,    0],
       [   0,    0,    0, ...,    0,    0,    0],
       [   1,    1,    1, ...,    0,    0,    0]])>, <tf.Tensor: shape=(), dtype=int32, numpy=3>)
(<tf.Tensor: shape=(3, 512), dtype=int32, numpy=
array([[  101,  3132, 10427, ...,     0,     0,     0],
       [    0,     0,     0, ...,     0,     0,     0],
       [    1,     1,     1, ...,     0,     0,     0]])>, <tf.Tensor: shape=(), dtype=int32, numpy=1>)
(<tf.Tensor: shape=(3, 512), dtype=int32, numpy=
array([[ 101, 3291, 2007, ...,    0,    0,    0],
       [   0,    0,    0, ...,    0,    0,    0],
       [   1,    1,    1, ...,    0,    0,    0]])>, <tf.Tensor: shape=(), dtype=int32, numpy=5>)
(<tf.Tensor: shape=(3, 512), dtype=int32, numpy=
array([[ 101, 3733, 1998, ...,    0,    0,    0],
       [   0,    0,    0, ...,    0,    0,    0],
       [   1,    1,    1, ...,    0,    0,    0]])>, <tf.Tensor: shape=(), dtype=int32, nump

(<tf.Tensor: shape=(3, 512), dtype=int32, numpy=
array([[ 101, 4152, 1996, ...,    0,    0,    0],
       [   0,    0,    0, ...,    0,    0,    0],
       [   1,    1,    1, ...,    0,    0,    0]])>, <tf.Tensor: shape=(), dtype=int32, numpy=2>)
(<tf.Tensor: shape=(3, 512), dtype=int32, numpy=
array([[ 101, 2515, 2025, ...,    0,    0,    0],
       [   0,    0,    0, ...,    0,    0,    0],
       [   1,    1,    1, ...,    0,    0,    0]])>, <tf.Tensor: shape=(), dtype=int32, numpy=2>)
(<tf.Tensor: shape=(3, 512), dtype=int32, numpy=
array([[ 101, 1045, 2293, ...,    0,    0,    0],
       [   0,    0,    0, ...,    0,    0,    0],
       [   1,    1,    1, ...,    0,    0,    0]])>, <tf.Tensor: shape=(), dtype=int32, numpy=5>)
(<tf.Tensor: shape=(3, 512), dtype=int32, numpy=
array([[ 101, 1045, 2293, ...,    0,    0,    0],
       [   0,    0,    0, ...,    0,    0,    0],
       [   1,    1,    1, ...,    0,    0,    0]])>, <tf.Tensor: shape=(), dtype=int32, numpy=5>)
(<tf.Tensor:

(<tf.Tensor: shape=(3, 512), dtype=int32, numpy=
array([[  101,  2053, 11924, ...,     0,     0,     0],
       [    0,     0,     0, ...,     0,     0,     0],
       [    1,     1,     1, ...,     0,     0,     0]])>, <tf.Tensor: shape=(), dtype=int32, numpy=1>)
(<tf.Tensor: shape=(3, 512), dtype=int32, numpy=
array([[  101,  3835, 10439, ...,     0,     0,     0],
       [    0,     0,     0, ...,     0,     0,     0],
       [    1,     1,     1, ...,     0,     0,     0]])>, <tf.Tensor: shape=(), dtype=int32, numpy=4>)
(<tf.Tensor: shape=(3, 512), dtype=int32, numpy=
array([[ 101, 2307, 2005, ...,    0,    0,    0],
       [   0,    0,    0, ...,    0,    0,    0],
       [   1,    1,    1, ...,    0,    0,    0]])>, <tf.Tensor: shape=(), dtype=int32, numpy=1>)
(<tf.Tensor: shape=(3, 512), dtype=int32, numpy=
array([[ 101, 2025, 1037, ...,    0,    0,    0],
       [   0,    0,    0, ...,    0,    0,    0],
       [   1,    1,    1, ...,    0,    0,    0]])>, <tf.Tensor: shape=(),

(<tf.Tensor: shape=(3, 512), dtype=int32, numpy=
array([[ 101, 2044, 2244, ...,    0,    0,    0],
       [   0,    0,    0, ...,    0,    0,    0],
       [   1,    1,    1, ...,    0,    0,    0]])>, <tf.Tensor: shape=(), dtype=int32, numpy=4>)
(<tf.Tensor: shape=(3, 512), dtype=int32, numpy=
array([[ 101, 2200, 3835, ...,    0,    0,    0],
       [   0,    0,    0, ...,    0,    0,    0],
       [   1,    1,    1, ...,    0,    0,    0]])>, <tf.Tensor: shape=(), dtype=int32, numpy=4>)
(<tf.Tensor: shape=(3, 512), dtype=int32, numpy=
array([[ 101, 2307, 2173, ...,    0,    0,    0],
       [   0,    0,    0, ...,    0,    0,    0],
       [   1,    1,    1, ...,    0,    0,    0]])>, <tf.Tensor: shape=(), dtype=int32, numpy=3>)
(<tf.Tensor: shape=(3, 512), dtype=int32, numpy=
array([[ 101, 1045, 5223, ...,    0,    0,    0],
       [   0,    0,    0, ...,    0,    0,    0],
       [   1,    1,    1, ...,    0,    0,    0]])>, <tf.Tensor: shape=(), dtype=int32, numpy=1>)
(<tf.Tensor:

...

(<tf.Tensor: shape=(3, 512), dtype=int32, numpy=
array([[ 101, 3819, 2005, ...,    0,    0,    0],
       [   0,    0,    0, ...,    0,    0,    0],
       [   1,    1,    1, ...,    0,    0,    0]])>, <tf.Tensor: shape=(), dtype=int32, numpy=4>)
(<tf.Tensor: shape=(3, 512), dtype=int32, numpy=
array([[ 101, 1045, 2428, ...,    0,    0,    0],
       [   0,    0,    0, ...,    0,    0,    0],
       [   1,    1,    1, ...,    0,    0,    0]])>, <tf.Tensor: shape=(), dtype=int32, numpy=4>)
(<tf.Tensor: shape=(3, 512), dtype=int32, numpy=
array([[ 101, 1045, 2428, ...,    0,    0,    0],
       [   0,    0,    0, ...,    0,    0,    0],
       [   1,    1,    1, ...,    0,    0,    0]])>, <tf.Tensor: shape=(), dtype=int32, numpy=4>)
(<tf.Tensor: shape=(3, 512), dtype=int32, numpy=
array([[ 101, 2035, 2157, ...,    0,    0,    0],
       [   0,    0,    0, ...,    0,    0,    0],
       [   1,    1,    1, ...,    0,    0,    0]])>, <tf.Tensor: shape=(), dtype=int32, numpy=2>)
(<tf.Tensor:

(<tf.Tensor: shape=(3, 512), dtype=int32, numpy=
array([[ 101, 4550, 3722, ...,    0,    0,    0],
       [   0,    0,    0, ...,    0,    0,    0],
       [   1,    1,    1, ...,    0,    0,    0]])>, <tf.Tensor: shape=(), dtype=int32, numpy=4>)
(<tf.Tensor: shape=(3, 512), dtype=int32, numpy=
array([[  101, 10651,  3333, ...,     0,     0,     0],
       [    0,     0,     0, ...,     0,     0,     0],
       [    1,     1,     1, ...,     0,     0,     0]])>, <tf.Tensor: shape=(), dtype=int32, numpy=3>)
(<tf.Tensor: shape=(3, 512), dtype=int32, numpy=
array([[ 101, 1045, 2066, ...,    0,    0,    0],
       [   0,    0,    0, ...,    0,    0,    0],
       [   1,    1,    1, ...,    0,    0,    0]])>, <tf.Tensor: shape=(), dtype=int32, numpy=4>)
(<tf.Tensor: shape=(3, 512), dtype=int32, numpy=
array([[ 101, 2070, 3492, ...,    0,    0,    0],
       [   0,    0,    0, ...,    0,    0,    0],
       [   1,    1,    1, ...,    0,    0,    0]])>, <tf.Tensor: shape=(), dtype=int32, nump

(<tf.Tensor: shape=(3, 512), dtype=int32, numpy=
array([[ 101, 2000, 3193, ...,    0,    0,    0],
       [   0,    0,    0, ...,    0,    0,    0],
       [   1,    1,    1, ...,    0,    0,    0]])>, <tf.Tensor: shape=(), dtype=int32, numpy=4>)
(<tf.Tensor: shape=(3, 512), dtype=int32, numpy=
array([[  101,  2023, 10439, ...,     0,     0,     0],
       [    0,     0,     0, ...,     0,     0,     0],
       [    1,     1,     1, ...,     0,     0,     0]])>, <tf.Tensor: shape=(), dtype=int32, numpy=1>)
(<tf.Tensor: shape=(3, 512), dtype=int32, numpy=
array([[ 101, 2001, 2307, ...,    0,    0,    0],
       [   0,    0,    0, ...,    0,    0,    0],
       [   1,    1,    1, ...,    0,    0,    0]])>, <tf.Tensor: shape=(), dtype=int32, numpy=4>)
(<tf.Tensor: shape=(3, 512), dtype=int32, numpy=
array([[  101,  1996, 10439, ...,     0,     0,     0],
       [    0,     0,     0, ...,     0,     0,     0],
       [    1,     1,     1, ...,     0,     0,     0]])>, <tf.Tensor: shape=(),

(<tf.Tensor: shape=(3, 512), dtype=int32, numpy=
array([[  101, 26351,  2515, ...,     0,     0,     0],
       [    0,     0,     0, ...,     0,     0,     0],
       [    1,     1,     1, ...,     0,     0,     0]])>, <tf.Tensor: shape=(), dtype=int32, numpy=4>)
(<tf.Tensor: shape=(3, 512), dtype=int32, numpy=
array([[ 101, 1045, 7078, ...,    0,    0,    0],
       [   0,    0,    0, ...,    0,    0,    0],
       [   1,    1,    1, ...,    0,    0,    0]])>, <tf.Tensor: shape=(), dtype=int32, numpy=4>)
(<tf.Tensor: shape=(3, 512), dtype=int32, numpy=
array([[ 101, 1996, 2732, ...,    0,    0,    0],
       [   0,    0,    0, ...,    0,    0,    0],
       [   1,    1,    1, ...,    0,    0,    0]])>, <tf.Tensor: shape=(), dtype=int32, numpy=2>)
(<tf.Tensor: shape=(3, 512), dtype=int32, numpy=
array([[ 101, 1045, 2428, ...,    0,    0,    0],
       [   0,    0,    0, ...,    0,    0,    0],
       [   1,    1,    1, ...,    0,    0,    0]])>, <tf.Tensor: shape=(), dtype=int32, nump

(<tf.Tensor: shape=(3, 512), dtype=int32, numpy=
array([[ 101, 2053, 2126, ...,    0,    0,    0],
       [   0,    0,    0, ...,    0,    0,    0],
       [   1,    1,    1, ...,    0,    0,    0]])>, <tf.Tensor: shape=(), dtype=int32, numpy=3>)
(<tf.Tensor: shape=(3, 512), dtype=int32, numpy=
array([[ 101, 1037, 2200, ...,    0,    0,    0],
       [   0,    0,    0, ...,    0,    0,    0],
       [   1,    1,    1, ...,    0,    0,    0]])>, <tf.Tensor: shape=(), dtype=int32, numpy=1>)
(<tf.Tensor: shape=(3, 512), dtype=int32, numpy=
array([[ 101, 9826, 1996, ...,    0,    0,    0],
       [   0,    0,    0, ...,    0,    0,    0],
       [   1,    1,    1, ...,    0,    0,    0]])>, <tf.Tensor: shape=(), dtype=int32, numpy=1>)
(<tf.Tensor: shape=(3, 512), dtype=int32, numpy=
array([[  101,  2023, 10439, ...,     0,     0,     0],
       [    0,     0,     0, ...,     0,     0,     0],
       [    1,     1,     1, ...,     0,     0,     0]])>, <tf.Tensor: shape=(), dtype=int32, nump

In [168]:
embbeded = [tokenizer(sent, return_tensors='pt', max_length=512, truncation=True, padding='max_length') for sent in clean_df]
bert_layer = transformers.AutoModel.from_pretrained('bert-base-uncased')
bert_layer(embbeded[0]['input_ids'],embbeded[0]['token_type_ids'],embbeded[0]['attention_mask'])
#print(embbeded[0:5])

dict_items([('input_ids', tensor([[  101,  1045,  3685,  2330,  1996, 10439,  4902,   102,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,    

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


BaseModelOutputWithPoolingAndCrossAttentions(last_hidden_state=tensor([[[-0.1581,  0.6537,  0.0198,  ..., -0.9273,  0.4994, -0.5532],
         [-0.2396,  0.6874, -0.0107,  ..., -0.8771,  0.5176, -0.5603],
         [-0.0678,  0.6506, -0.0054,  ..., -0.8207,  0.5638, -0.6236],
         ...,
         [-0.0852,  0.6840,  0.0870,  ..., -0.8950,  0.4701, -0.8727],
         [-0.0887,  0.6632,  0.0804,  ..., -0.9093,  0.4783, -0.8999],
         [-0.0765,  0.7212,  0.0939,  ..., -0.9929,  0.4941, -0.8954]]],
       grad_fn=<NativeLayerNormBackward0>), pooler_output=tensor([[ 4.0134e-02, -2.8189e-01, -6.7049e-01,  4.6019e-01,  7.9795e-01,
         -1.9980e-01, -4.5383e-01,  8.3236e-02, -8.9950e-01,  9.2613e-01,
         -5.0168e-01,  9.7313e-01,  3.4032e-01,  2.0065e-01,  5.3844e-01,
          1.6991e-02, -8.1761e-01,  3.7152e-01,  3.7837e-01,  7.9699e-01,
          7.7599e-01,  9.9971e-01, -5.8290e-01,  1.0901e-01,  2.6038e-01,
          9.9311e-01, -8.6177e-01,  5.3618e-01,  2.7199e-01,  5.403

torch.Size([1, 15])


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


[({'input_ids': tensor([[ 101, 2200, 2204, 8389,  999,  102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1]])},
  4),
 ({'input_ids': tensor([[  101,  2190, 10439,  2005,  5292,  2978,  4325,  1045,  3866,  2009,
            3565,   102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])},
  5),
 ({'input_ids': tensor([[  101, 10651,  2044,  2086,  1998,  6197,  1997,  2026, 12247,  4311,
            2006,  3248, 19277,  5796, 12247,  1998, 10474,  2017,  2145,  4033,
            1005,  1056,  9611,  1996,  2364,  3291,  2023, 10439,  2038,  4281,
           26351,  2987,  1005,  1056,  2147,  1012,  1012,  1012,  2024,  2017,
           12489,  2033,  1029,  1996, 10439,  2038,  2019, 12476,  2640,  2021,
            2302,  4281, 14764, 26351,  2003,  3432, 11809,  1045,  1005,  2310,
            2288,  1037,  2028, 24759,  2271,  2021,  2673,  3272,  202

In [175]:
BATCH_SIZE = 32
all_batched = all_dataset.padded_batch(BATCH_SIZE, padded_shapes=((3, None), ()),
                                       padding_values=(0, 0))
for i in all_batched:
    print(i)
NB_BATCHES = len(sorted_all) // BATCH_SIZE
NB_BATCHES

(<tf.Tensor: shape=(32, 3, 512), dtype=int32, numpy=
array([[[ 101, 4907, 5798, ...,    0,    0,    0],
        [   0,    0,    0, ...,    0,    0,    0],
        [   1,    1,    1, ...,    0,    0,    0]],

       [[ 101, 2200, 6179, ...,    0,    0,    0],
        [   0,    0,    0, ...,    0,    0,    0],
        [   1,    1,    1, ...,    0,    0,    0]],

       [[ 101, 3733, 2000, ...,    0,    0,    0],
        [   0,    0,    0, ...,    0,    0,    0],
        [   1,    1,    1, ...,    0,    0,    0]],

       ...,

       [[ 101, 2200, 3161, ...,    0,    0,    0],
        [   0,    0,    0, ...,    0,    0,    0],
        [   1,    1,    1, ...,    0,    0,    0]],

       [[ 101, 5263, 2000, ...,    0,    0,    0],
        [   0,    0,    0, ...,    0,    0,    0],
        [   1,    1,    1, ...,    0,    0,    0]],

       [[ 101, 3492, 2204, ...,    0,    0,    0],
        [   0,    0,    0, ...,    0,    0,    0],
        [   1,    1,    1, ...,    0,    0,    0]]])>, <t

(<tf.Tensor: shape=(32, 3, 512), dtype=int32, numpy=
array([[[  101,  3733,  2000, ...,     0,     0,     0],
        [    0,     0,     0, ...,     0,     0,     0],
        [    1,     1,     1, ...,     0,     0,     0]],

       [[  101,  2061,  2521, ...,     0,     0,     0],
        [    0,     0,     0, ...,     0,     0,     0],
        [    1,     1,     1, ...,     0,     0,     0]],

       [[  101,  2009,  2987, ...,     0,     0,     0],
        [    0,     0,     0, ...,     0,     0,     0],
        [    1,     1,     1, ...,     0,     0,     0]],

       ...,

       [[  101,  2009,  3271, ...,     0,     0,     0],
        [    0,     0,     0, ...,     0,     0,     0],
        [    1,     1,     1, ...,     0,     0,     0]],

       [[  101,  2307,  2126, ...,     0,     0,     0],
        [    0,     0,     0, ...,     0,     0,     0],
        [    1,     1,     1, ...,     0,     0,     0]],

       [[  101,  2204, 10439, ...,     0,     0,     0],
        [   

(<tf.Tensor: shape=(32, 3, 512), dtype=int32, numpy=
array([[[  101,  2342,  2062, ...,     0,     0,     0],
        [    0,     0,     0, ...,     0,     0,     0],
        [    1,     1,     1, ...,     0,     0,     0]],

       [[  101,  6581,  1012, ...,     0,     0,     0],
        [    0,     0,     0, ...,     0,     0,     0],
        [    1,     1,     1, ...,     0,     0,     0]],

       [[  101, 10047,  2061, ...,     0,     0,     0],
        [    0,     0,     0, ...,     0,     0,     0],
        [    1,     1,     1, ...,     0,     0,     0]],

       ...,

       [[  101,  3722,  2000, ...,     0,     0,     0],
        [    0,     0,     0, ...,     0,     0,     0],
        [    1,     1,     1, ...,     0,     0,     0]],

       [[  101,  2145,  2000, ...,     0,     0,     0],
        [    0,     0,     0, ...,     0,     0,     0],
        [    1,     1,     1, ...,     0,     0,     0]],

       [[  101,  7906,  2033, ...,     0,     0,     0],
        [   

(<tf.Tensor: shape=(32, 3, 512), dtype=int32, numpy=
array([[[ 101, 8909, 2243, ...,    0,    0,    0],
        [   0,    0,    0, ...,    0,    0,    0],
        [   1,    1,    1, ...,    0,    0,    0]],

       [[ 101, 2200, 3937, ...,    0,    0,    0],
        [   0,    0,    0, ...,    0,    0,    0],
        [   1,    1,    1, ...,    0,    0,    0]],

       [[ 101, 3722, 3733, ...,    0,    0,    0],
        [   0,    0,    0, ...,    0,    0,    0],
        [   1,    1,    1, ...,    0,    0,    0]],

       ...,

       [[ 101, 7126, 2007, ...,    0,    0,    0],
        [   0,    0,    0, ...,    0,    0,    0],
        [   1,    1,    1, ...,    0,    0,    0]],

       [[ 101, 1045, 2228, ...,    0,    0,    0],
        [   0,    0,    0, ...,    0,    0,    0],
        [   1,    1,    1, ...,    0,    0,    0]],

       [[ 101, 2061, 2521, ...,    0,    0,    0],
        [   0,    0,    0, ...,    0,    0,    0],
        [   1,    1,    1, ...,    0,    0,    0]]])>, <t

(<tf.Tensor: shape=(32, 3, 512), dtype=int32, numpy=
array([[[  101,  2126,  2058, ...,     0,     0,     0],
        [    0,     0,     0, ...,     0,     0,     0],
        [    1,     1,     1, ...,     0,     0,     0]],

       [[  101,  3791,  2147, ...,     0,     0,     0],
        [    0,     0,     0, ...,     0,     0,     0],
        [    1,     1,     1, ...,     0,     0,     0]],

       [[  101,  2515,  2025, ...,     0,     0,     0],
        [    0,     0,     0, ...,     0,     0,     0],
        [    1,     1,     1, ...,     0,     0,     0]],

       ...,

       [[  101, 12476, 10439, ...,     0,     0,     0],
        [    0,     0,     0, ...,     0,     0,     0],
        [    1,     1,     1, ...,     0,     0,     0]],

       [[  101,  4895,  7076, ...,     0,     0,     0],
        [    0,     0,     0, ...,     0,     0,     0],
        [    1,     1,     1, ...,     0,     0,     0]],

       [[  101,  4149,  2440, ...,     0,     0,     0],
        [   

...

(<tf.Tensor: shape=(32, 3, 512), dtype=int32, numpy=
array([[[  101,  1045,  2031, ...,     0,     0,     0],
        [    0,     0,     0, ...,     0,     0,     0],
        [    1,     1,     1, ...,     0,     0,     0]],

       [[  101,  1045,  1005, ...,     0,     0,     0],
        [    0,     0,     0, ...,     0,     0,     0],
        [    1,     1,     1, ...,     0,     0,     0]],

       [[  101,  3025,  3277, ...,     0,     0,     0],
        [    0,     0,     0, ...,     0,     0,     0],
        [    1,     1,     1, ...,     0,     0,     0]],

       ...,

       [[  101,  1045,  2066, ...,     0,     0,     0],
        [    0,     0,     0, ...,     0,     0,     0],
        [    1,     1,     1, ...,     0,     0,     0]],

       [[  101,  2023, 10439, ...,     0,     0,     0],
        [    0,     0,     0, ...,     0,     0,     0],
        [    1,     1,     1, ...,     0,     0,     0]],

       [[  101,  3444,  5227, ...,     0,     0,     0],
        [   

(<tf.Tensor: shape=(32, 3, 512), dtype=int32, numpy=
array([[[  101,  2428,  4487, ...,     0,     0,     0],
        [    0,     0,     0, ...,     0,     0,     0],
        [    1,     1,     1, ...,     0,     0,     0]],

       [[  101,  1045,  2031, ...,     0,     0,     0],
        [    0,     0,     0, ...,     0,     0,     0],
        [    1,     1,     1, ...,     0,     0,     0]],

       [[  101,  1045,  8840, ...,     0,     0,     0],
        [    0,     0,     0, ...,     0,     0,     0],
        [    1,     1,     1, ...,     0,     0,     0]],

       ...,

       [[  101,  2023, 10439, ...,     0,     0,     0],
        [    0,     0,     0, ...,     0,     0,     0],
        [    1,     1,     1, ...,     0,     0,     0]],

       [[  101,  3432,  2293, ...,     0,     0,     0],
        [    0,     0,     0, ...,     0,     0,     0],
        [    1,     1,     1, ...,     0,     0,     0]],

       [[  101,  2123,  2102, ...,     0,     0,     0],
        [   

(<tf.Tensor: shape=(32, 3, 512), dtype=int32, numpy=
array([[[  101,  3271,  2000, ...,     0,     0,     0],
        [    0,     0,     0, ...,     0,     0,     0],
        [    1,     1,     1, ...,     0,     0,     0]],

       [[  101,  2023,  2003, ...,     0,     0,     0],
        [    0,     0,     0, ...,     0,     0,     0],
        [    1,     1,     1, ...,     0,     0,     0]],

       [[  101,  4149,  1996, ...,     0,     0,     0],
        [    0,     0,     0, ...,     0,     0,     0],
        [    1,     1,     1, ...,     0,     0,     0]],

       ...,

       [[  101, 24501,  5644, ...,     0,     0,     0],
        [    0,     0,     0, ...,     0,     0,     0],
        [    1,     1,     1, ...,     0,     0,     0]],

       [[  101,  2023, 10439, ...,     0,     0,     0],
        [    0,     0,     0, ...,     0,     0,     0],
        [    1,     1,     1, ...,     0,     0,     0]],

       [[  101,  2009,  2052, ...,     0,     0,     0],
        [   

(<tf.Tensor: shape=(32, 3, 512), dtype=int32, numpy=
array([[[  101,  2043,  2017, ...,     0,     0,     0],
        [    0,     0,     0, ...,     0,     0,     0],
        [    1,     1,     1, ...,     0,     0,     0]],

       [[  101,  1045,  1005, ...,     0,     0,     0],
        [    0,     0,     0, ...,     0,     0,     0],
        [    1,     1,     1, ...,     0,     0,     0]],

       [[  101,  2061,  2521, ...,     0,     0,     0],
        [    0,     0,     0, ...,     0,     0,     0],
        [    1,     1,     1, ...,     0,     0,     0]],

       ...,

       [[  101,  4013,  5310, ...,     0,     0,     0],
        [    0,     0,     0, ...,     0,     0,     0],
        [    1,     1,     1, ...,     0,     0,     0]],

       [[  101, 10140,  1998, ...,     0,     0,     0],
        [    0,     0,     0, ...,     0,     0,     0],
        [    1,     1,     1, ...,     0,     0,     0]],

       [[  101,  2023,  2003, ...,     0,     0,     0],
        [   

349

In [176]:
NB_BATCHES_TEST = NB_BATCHES // 10
NB_BATCHES_TEST

34

In [191]:
all_batched.shuffle(NB_BATCHES)
test_dataset = all_batched.take(NB_BATCHES_TEST)
train_dataset = all_batched.skip(NB_BATCHES_TEST)
for i in train_dataset:
    print(i[:, 0, :])

TypeError: tuple indices must be integers or slices, not tuple

In [187]:
from tensorflow.keras import layers

class DCNN(tf.keras.Model):

    def __init__(self,
               
               nb_filters = 50,
               FFN_units=512,
               nb_classes=2,
               dropout_rate=0.1,
               training=False,
               name="dcnn"):
        super(DCNN, self).__init__(name=name)

        self.bert_layer = AutoModel.from_pretrained('bert-base-uncased')

        self.bigram = layers.Conv1D(filters = nb_filters,
                                    kernel_size = 2,
                                    padding='valid',
                                    activation='relu')
        self.trigram = layers.Conv1D(filters = nb_filters,
                                    kernel_size = 3,
                                    padding='valid',
                                    activation='relu')
        self.fourgram = layers.Conv1D(filters = nb_filters,
                                    kernel_size = 4,
                                    padding='valid',
                                    activation='relu')

        self.pool = layers.GlobalMaxPool1D()

        self.dense_1 = layers.Dense(units = FFN_units, activation='relu')
        self.dropout = layers.Dropout(rate=dropout_rate)
        if nb_classes == 2:
            self.last_dense = layers.Dense(units=1, activation='sigmoid')
        else:
            self.last_dense = layers.Dense(units=nb_classes, activation='softmax')

    def embed_with_bert(self, all_tokens):
        for i in all_tokens:
            print(i)
        _, embs = self.bert_layer([all_tokens[:, 0, :],
                                 all_tokens[:, 1, :],
                                 all_tokens[:, 2, :]])
        return embs
    def call(self, inputs, training):
        x = self.embed_with_bert(inputs)
        x_1 = self.bigram(x)
        x_1 = self.pool(x_1)
        x_2 = self.trigram(x)
        x_2 = self.pool(x_2)
        x_3 = self.fourgram(x)
        x_3 = self.pool(x_3)

        merged = tf.concat([x_1, x_2, x_3], axis = -1)
        merged = self.dense_1(merged)
        merged = self.dropout(merged, training)
        output = self.last_dense(merged)

        return output

In [188]:
VOCAB_SIZE = len(tokenizer.vocab)
#EMB_DIM = 200
NB_FILTERS = 100
FFN_UNITS = 256
NB_CLASSES = 2
DROPOUT_RATE = 0.2
NB_EPOCHS = 5

print(VOCAB_SIZE)

30522


In [189]:
Dcnn = DCNN(
            nb_filters = NB_FILTERS,
            FFN_units = FFN_UNITS,
            nb_classes = NB_CLASSES,
            dropout_rate = DROPOUT_RATE)
if NB_CLASSES == 2:
    Dcnn.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
else:
    Dcnn.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['sparse_categorical_accuracy'])

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [135]:
ckpt = tf.train.Checkpoint(Dcnn=Dcnn)

ckpt_manager = tf.train.CheckpointManager(ckpt, 'samples/', max_to_keep=1)

if ckpt_manager.latest_checkpoint:
    ckpt.restore(ckpt_manager.latest_checkpoint)
    print('Latest checkpoint restored!')
    
class MyCustomCallBack(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        ckpt_manager.save()
        print("Checkpoint savet at samples/")

In [190]:
history = Dcnn.fit(train_dataset,
                   epochs=NB_EPOCHS)#,
                   #steps_per_epoch = 100,
                   #callbacks=[MyCustomCallBack()])

Epoch 1/5
Tensor("dcnn/while/TensorArrayV2Read/TensorListGetItem:0", shape=(3, None), dtype=int32)


AttributeError: in user code:

    File "C:\Users\realmeid\Anaconda3\envs\py37\lib\site-packages\keras\engine\training.py", line 1051, in train_function  *
        return step_function(self, iterator)
    File "C:\Users\realmeid\Anaconda3\envs\py37\lib\site-packages\keras\engine\training.py", line 1040, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "C:\Users\realmeid\Anaconda3\envs\py37\lib\site-packages\keras\engine\training.py", line 1030, in run_step  **
        outputs = model.train_step(data)
    File "C:\Users\realmeid\Anaconda3\envs\py37\lib\site-packages\keras\engine\training.py", line 889, in train_step
        y_pred = self(x, training=True)
    File "C:\Users\realmeid\Anaconda3\envs\py37\lib\site-packages\keras\utils\traceback_utils.py", line 67, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "C:\Users\realmeid\AppData\Local\Temp\__autograph_generated_fileowp38zy9.py", line 10, in tf__ca

In [50]:
history.history.keys()

results = Dcnn.evaluate(test_dataset)
print(results)

def get_prediction(sentence):
  tokens = tokenizer.convert_tokens_to_ids(tokenizer.tokenize(sentence))
  inputs = tf.expand_dims(tokens, 0) # (batch_size) (1,...)

  output = Dcnn(inputs, training=False)

  sentiment = math.floor(output*2)
  print(sentiment)
  if sentiment == 0:
    print('negative')
  elif sentiment == 1:
    print('positive')

35/35 [==============================] - 0s 4ms/step - loss: -3457844992.0000 - accuracy: 0.1634
[-3457844992.0, 0.16339285671710968]


In [51]:
get_prediction('This movie was pretty interesting')

2
